In [13]:
import pandas as pd
import os
import VCF

In [14]:
def count_comments(filename):
    """ Count comment lines (those that start with "#") 
        cribbed from slowko """
    comments = 0
    fn_open = gzip.open if filename.endswith('.gz') else open
    with fn_open(filename) as fh:
        for line in fh:
            if line.startswith('#'):
                comments += 1
            else:
                break
    return comments



def vcf_to_dataframe(filename):
    """ Open a VCF file and return a pandas.DataFrame with
        each INFO field included as a column in the dataframe 
        cribbed from slowko """
    VCF_HEADER = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', '20']

    # Count how many comment lines should be skipped.
    comments = count_comments(filename)
    tbl = pd.read_table(filename, compression=None, skiprows=comments,
                        names=VCF_HEADER, usecols=range(10))

    return(tbl)



def coverage_search_on_vcf(df):
    """ given subset of vcf entries, search for AD (DepthPerAlleleBySample) col """ 
    counts = ''
    for i in range(0, len(df.index)):
        row = df.iloc[i]
        extra_col = str(row['20'])

        try:
            AD = extra_col.split(':')[1]
            wt_count = int(AD.split(',')[0])
            variant_count = int(AD.split(',')[1])
            total_count = wt_count + variant_count

            ratio = str(variant_count) + ':' + str(total_count)
            counts = ratio

        except: # picking up a wierd edge case -- '20' field is malformed
            print(extra_col)
            continue

    return(counts)

In [15]:
! aws s3 cp s3://darmanis-group/lincoln_scratch/LAUD_revisions/vcf_3.13.20/A10_1001000329.vcf . 

download: s3://darmanis-group/lincoln_scratch/LAUD_revisions/vcf_3.13.20/A10_1001000329.vcf to ./A10_1001000329.vcf


In [17]:
! aws s3 cp s3://darmanis-group/singlecell_lungadeno/non_immune/nonImmune_bams_9.27/gVCF/A10_1001000407.g.vcf . 

download: s3://darmanis-group/singlecell_lungadeno/non_immune/nonImmune_bams_9.27/gVCF/A10_1001000407.g.vcf to ./A10_1001000407.g.vcf


In [19]:
vcf_df = vcf_to_dataframe('A10_1001000329.vcf')
gvcf_df = vcf_to_dataframe('A10_1001000407.g.vcf')

In [20]:
vcf_df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,20
0,chr1,135804,.,G,A,99.10,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,4:4:12:113,12,0"
1,chr1,135860,.,G,GCCC,78.32,.,AC=2;AF=1.00;AN=2;DP=10;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:90,6,0"
2,chr1,135863,.,A,ATGTACTCTGCGTTGAT,78.32,.,AC=2;AF=1.00;AN=2;DP=9;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:90,6,0"
3,chr1,632403,.,T,C,37.28,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:49,6,0"
4,chr1,632427,rs2853819,T,C,37.28,.,AC=2;AF=1.00;AN=2;DB;DP=2;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:49,6,0"
...,...,...,...,...,...,...,...,...,...,...
7423,ERCC-00145,559,.,A,T,890.60,.,AC=1;AF=0.500;AN=2;DP=174;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"0/1:135,39:174:99:898,0,4667"
7424,ERCC-00145,707,.,G,A,679.60,.,AC=1;AF=0.500;AN=2;DP=121;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"0/1:94,26:120:99:687,0,3217"
7425,ERCC-00145,729,.,A,G,562.60,.,AC=1;AF=0.500;AN=2;DP=127;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"0/1:101,25:126:99:570,0,3380"
7426,ERCC-00170,257,.,T,A,964.03,.,AC=2;AF=1.00;AN=2;DP=24;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,24:24:72:978,72,0"


In [21]:
# looks like these are laid out the same way 
gvcf_df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,20
0,chr1,631811,.,C,T,68.28,.,AC=2;AF=1.00;AN=2;DP=30;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:80,6,0"
1,chr1,631861,.,G,A,99.60,.,AC=1;AF=0.500;AN=2;DP=110;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"0/1:29,5:34:99:107,0,1012"
2,chr1,631862,.,G,A,1010.03,.,AC=2;AF=1.00;AN=2;DP=113;ExcessHet=3.0103;FS=7...,GT:AD:DP:GQ:PL,"1/1:1,33:34:72:1024,72,0"
3,chr1,632199,.,C,T,654.03,.,AC=2;AF=1.00;AN=2;DP=1244;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:0,16:16:48:668,48,0"
4,chr1,632403,.,T,C,35.44,.,AC=2;AF=1.00;AN=2;DP=8;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:45,3,0"
...,...,...,...,...,...,...,...,...,...,...
6347,chrM,15326,.,A,G,71989.03,.,AC=2;AF=1.00;AN=2;DP=1787;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:11,1775:1786:99:72003,4930,0"
6348,chrM,15887,.,T,G,38.94,.,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:5:50,5,0"
6349,chrM,15888,.,GTCCTTGTAGTAT,G,35.66,.,AC=2;AF=1.00;AN=2;DP=1;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:4:46,4,0"
6350,chrM,16311,.,T,C,1086.03,.,AC=2;AF=1.00;AN=2;DP=27;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,27:27:81:1100,81,0"


In [22]:
#////////////////////////////////////////////////////////////////////
#////////////////////////////////////////////////////////////////////
#////////////////////////////////////////////////////////////////////

In [23]:
# want to comp for the exact same cell 
! aws s3 cp s3://darmanis-group/singlecell_lungadeno/non_immune/nonImmune_bams_9.27/vcf1/A10_1001000407.vcf . 

download: s3://darmanis-group/singlecell_lungadeno/non_immune/nonImmune_bams_9.27/vcf1/A10_1001000407.vcf to ./A10_1001000407.vcf


In [24]:
vcf_df = vcf_to_dataframe('A10_1001000407.vcf')

In [25]:
vcf_df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,20
0,chr1,631811,.,C,T,52.74,.,AC=2;AF=1.00;AN=2;DP=30;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:80,6,0"
1,chr1,631861,.,G,A,78.77,.,AC=1;AF=0.500;AN=2;DP=110;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"0/1:29,5:34:99:107,0,1012"
2,chr1,631862,.,G,A,995.77,.,AC=2;AF=1.00;AN=2;DP=113;ExcessHet=3.0103;FS=7...,GT:AD:DP:GQ:PL,"1/1:1,33:34:72:1024,72,0"
3,chr1,632199,.,C,T,639.77,.,AC=2;AF=1.00;AN=2;DP=1244;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:0,16:16:48:668,48,0"
4,chr1,632403,.,T,C,18.59,.,AC=2;AF=1.00;AN=2;DP=8;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:45,3,0"
...,...,...,...,...,...,...,...,...,...,...
6274,chrM,15326,.,A,G,72234.77,.,AC=2;AF=1.00;AN=2;DP=1786;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:9,1777:1786:99:72263,5013,0"
6275,chrM,15887,.,T,G,22.98,.,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:5:50,5,0"
6276,chrM,15888,.,GTCCTTGTAGTAT,G,10.58,.,AC=2;AF=1.00;AN=2;DP=1;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:4:46,4,0"
6277,chrM,16311,.,T,C,1071.77,.,AC=2;AF=1.00;AN=2;DP=27;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,27:27:81:1100,81,0"


In [26]:
# gvcf has more hits
gvcf_df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,20
0,chr1,631811,.,C,T,68.28,.,AC=2;AF=1.00;AN=2;DP=30;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:80,6,0"
1,chr1,631861,.,G,A,99.60,.,AC=1;AF=0.500;AN=2;DP=110;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"0/1:29,5:34:99:107,0,1012"
2,chr1,631862,.,G,A,1010.03,.,AC=2;AF=1.00;AN=2;DP=113;ExcessHet=3.0103;FS=7...,GT:AD:DP:GQ:PL,"1/1:1,33:34:72:1024,72,0"
3,chr1,632199,.,C,T,654.03,.,AC=2;AF=1.00;AN=2;DP=1244;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:0,16:16:48:668,48,0"
4,chr1,632403,.,T,C,35.44,.,AC=2;AF=1.00;AN=2;DP=8;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:45,3,0"
...,...,...,...,...,...,...,...,...,...,...
6347,chrM,15326,.,A,G,71989.03,.,AC=2;AF=1.00;AN=2;DP=1787;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:11,1775:1786:99:72003,4930,0"
6348,chrM,15887,.,T,G,38.94,.,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:5:50,5,0"
6349,chrM,15888,.,GTCCTTGTAGTAT,G,35.66,.,AC=2;AF=1.00;AN=2;DP=1;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:4:46,4,0"
6350,chrM,16311,.,T,C,1086.03,.,AC=2;AF=1.00;AN=2;DP=27;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,27:27:81:1100,81,0"


In [29]:
# lets try another one
! aws s3 cp s3://darmanis-group/singlecell_lungadeno/non_immune/nonImmune_bams_9.27/gVCF/A10_1001000408.g.vcf . 
! aws s3 cp s3://darmanis-group/singlecell_lungadeno/non_immune/nonImmune_bams_9.27/vcf1/A10_1001000408.vcf . 

download: s3://darmanis-group/singlecell_lungadeno/non_immune/nonImmune_bams_9.27/gVCF/A10_1001000408.g.vcf to ./A10_1001000408.g.vcf
download: s3://darmanis-group/singlecell_lungadeno/non_immune/nonImmune_bams_9.27/vcf1/A10_1001000408.vcf to ./A10_1001000408.vcf


In [30]:
vcf_df = vcf_to_dataframe('A10_1001000408.vcf')
gvcf_df = vcf_to_dataframe('A10_1001000408.g.vcf')

In [31]:
vcf_df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,20
0,chr1,135040,.,T,C,56.74,.,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:84,6,0"
1,chr1,135163,rs374499151,C,T,56.74,.,AC=2;AF=1.00;AN=2;DB;DP=2;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:84,6,0"
2,chr1,631622,.,T,C,15.65,.,AC=2;AF=1.00;AN=2;DP=5;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:42,3,0"
3,chr1,631653,.,T,C,15.65,.,AC=2;AF=1.00;AN=2;DP=5;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:42,3,0"
4,chr1,631811,.,C,T,56.74,.,AC=2;AF=1.00;AN=2;DP=17;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:84,6,0"
...,...,...,...,...,...,...,...,...,...,...
21729,chrUn_KI270742v1,23958,.,G,A,107.28,.,AC=2;AF=1.00;AN=2;DP=3;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,3:3:9:135,9,0"
21730,chrUn_KI270742v1,24073,.,TCTGC,T,143.00,.,AC=2;AF=1.00;AN=2;DP=5;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,4:4:12:180,12,0"
21731,chrUn_KI270742v1,24088,.,G,A,44.74,.,AC=2;AF=1.00;AN=2;DP=3;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:72,6,0"
21732,chrUn_KI270742v1,107730,.,A,G,222.84,.,AC=2;AF=1.00;AN=2;DP=6;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,6:6:18:251,18,0"


In [32]:
# great, this guy also has more
gvcf_df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,20
0,chr1,135040,.,T,C,72.28,.,AC=2;AF=1.00;AN=2;DP=2;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:84,6,0"
1,chr1,135163,rs374499151,C,T,72.28,.,AC=2;AF=1.00;AN=2;DB;DP=2;ExcessHet=3.0103;FS=...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:84,6,0"
2,chr1,631622,.,T,C,32.44,.,AC=2;AF=1.00;AN=2;DP=5;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:42,3,0"
3,chr1,631653,.,T,C,32.44,.,AC=2;AF=1.00;AN=2;DP=5;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,1:1:3:42,3,0"
4,chr1,631811,.,C,T,72.28,.,AC=2;AF=1.00;AN=2;DP=17;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:84,6,0"
...,...,...,...,...,...,...,...,...,...,...
22102,chrUn_KI270742v1,24088,.,G,A,60.28,.,AC=2;AF=1.00;AN=2;DP=3;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,2:2:6:72,6,0"
22103,chrUn_KI270742v1,34983,.,C,CAGAGA,32.06,.,AC=1;AF=0.500;AN=2;DP=2;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"0/1:0,1:1:0:39,0,0"
22104,chrUn_KI270742v1,34984,.,T,TTTATTCGTTTCTTGTTGCTCTTGGAAACACCCAGAGG,32.06,.,AC=1;AF=0.500;AN=2;DP=2;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"0/1:0,1:1:0:39,0,0"
22105,chrUn_KI270742v1,107730,.,A,G,236.93,.,AC=2;AF=1.00;AN=2;DP=6;ExcessHet=3.0103;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,6:6:18:251,18,0"
